# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.

In [1]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-a07074eb52-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# History Workflow Series ("Ancient Civilization explorative search") 

Consider the following exploratory information need:

> Investigate the Ancient Egypt and the related archeological explorations and findings.

## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | occupation    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wd:Q5`| Human       | node |
| `wd:Q13444093`| Meidum Pyramid    | node |
| `wd:Q41127`| Poseidon   | node |





Also consider

```
wd:Q13444093 ?p ?obj .
```

is the BGP to retrieve all **properties of Meidum Pyramid**

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.



The workload should:


1. Identify the BGP for Ancient Egypt

2. Find the Ancient Egypt pharaohs born between 1500 BCE and 1000 BCE who had more than 8 child (the result set must be a list of triples pharaoh IRI,label and #children).

3. Find all the ancient civilizations. Return the IRI and name of the civilization and also their duration (in number of years) if it is available.

4. Identify the BGP for archeologist.

5. For each ancient civilization find the number of archeologists who worked on it (return also the civilizations with no archeologists) (the result set must be triples of civilization IRI, label and #archeologist).

6. Find the BGP for deity.

7. List all the twelve olympian greek deities (the result set must a list of 12 elements of couples of deity IRI and label).

8. Compare ancient Greece with ancient Egypt and find how many deities each one has.


## Task 1
Identify the BGP for Ancient Egypt

In [7]:
# query example
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q13444093 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P131'), ('name', 'located in the administrative territorial entity')]
[('p', 'http://www.wikidata.org/prop/direct/P138'), ('name', 'named after')]
[('p', 'http://www.wikidata.org/prop/direct/P1481'), ('name', 'vici.org ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1566'), ('name', 'GeoNames ID')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P186'), ('name', 'made from material')]
[('p', 'http://www.wikidata.org/prop/direct/P2044'), ('name', 'elevation above sea level')]
[('p', 'http://www.wikidata.org/prop/direct/P2048'), ('name', 'height')]
[('p', 'http://www.wikidata.org/prop/direct/P214'), ('name', 'VIAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P227'), ('name', 'GND ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2326'), ('name', 'GNS Unique Feature ID')]
[('p', 'http://www.wi

In [8]:
# Meidum Pyramid commissioner
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   # bind something
   wd:Q13444093 wdt:P88 ?o .
   # get the label
   ?o sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q189371'), ('name', 'Sneferu')]
[('o', 'http://www.wikidata.org/entity/Q220240'), ('name', 'Huni')]
2


In [9]:
# Sneferu details
queryString = """
SELECT DISTINCT ?p ?p_name ?o ?name
WHERE {
   # bind something
   wd:Q189371 ?p ?o .
   # get the label
   ?p sc:name ?p_name.
   ?o sc:name ?name.
}
LIMIT 40
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P40'), ('p_name', 'child'), ('o', 'http://www.wikidata.org/entity/Q122426'), ('name', 'Henutsen')]
[('p', 'http://www.wikidata.org/prop/direct/P40'), ('p_name', 'child'), ('o', 'http://www.wikidata.org/entity/Q161904'), ('name', 'Khufu')]
[('p', 'http://www.wikidata.org/prop/direct/P2348'), ('p_name', 'time period'), ('o', 'http://www.wikidata.org/entity/Q177819'), ('name', 'Old Kingdom of Egypt')]
[('p', 'http://www.wikidata.org/prop/direct/P53'), ('p_name', 'family'), ('o', 'http://www.wikidata.org/entity/Q211001'), ('name', 'Fourth Dynasty of Egypt')]
[('p', 'http://www.wikidata.org/prop/direct/P26'), ('p_name', 'spouse'), ('o', 'http://www.wikidata.org/entity/Q256515'), ('name', 'Hetepheres I')]
[('p', 'http://www.wikidata.org/prop/direct/P3373'), ('p_name', 'sibling'), ('o', 'http://www.wikidata.org/entity/Q256515'), ('name', 'Hetepheres I')]
[('p', 'http://www.wikidata.org/prop/direct/P40'), ('p_name', 'child'), ('o', 'http://ww

Final query for this task

In [10]:
# write your final query

queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   # bind something
   wd:Q189371 wdt:P27 ?o .
   # get the label
   ?o sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q11768'), ('name', 'Ancient Egypt')]
1


## Task 2
Find the Ancient Egypt pharaohs born between 1500 BCE and 1000 BCE who had more than 8 child (the result set must be a list of triples pharaoh IRI,label and #children).

In [11]:
# Pharaos with more than 8 child
queryString = """
SELECT DISTINCT ?pha ?p_name (count(?c) as ?c_num)
WHERE {
   # bind something
   ?pha wdt:P97 wd:Q37110 .
   ?pha wdt:P40 ?c .
   # get the label
   ?pha sc:name ?p_name.
}
GROUP BY ?pha ?p_name
HAVING(count(?c)>8)
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('pha', 'http://www.wikidata.org/entity/Q199861'), ('p_name', 'Khafra'), ('c_num', '9')]
[('pha', 'http://www.wikidata.org/entity/Q7222'), ('p_name', 'Ahmose I'), ('c_num', '9')]
[('pha', 'http://www.wikidata.org/entity/Q189371'), ('p_name', 'Sneferu'), ('c_num', '13')]
[('pha', 'http://www.wikidata.org/entity/Q161904'), ('p_name', 'Khufu'), ('c_num', '14')]
[('pha', 'http://www.wikidata.org/entity/Q168261'), ('p_name', 'Ptolemy I Soter'), ('c_num', '9')]
[('pha', 'http://www.wikidata.org/entity/Q44387'), ('p_name', 'Darius I of Persia'), ('c_num', '9')]
[('pha', 'http://www.wikidata.org/entity/Q1528'), ('p_name', 'Ramesses III'), ('c_num', '10')]
[('pha', 'http://www.wikidata.org/entity/Q1523'), ('p_name', 'Ramesses II'), ('c_num', '18')]
[('pha', 'http://www.wikidata.org/entity/Q31526'), ('p_name', 'Seqenenre Tao'), ('c_num', '9')]
[('pha', 'http://www.wikidata.org/entity/Q42606'), ('p_name', 'Amenhotep III'), ('c_num', '10')]
[('pha', 'http://www.wikidata.org/entity/Q81794'

In [12]:
# Check query
queryString = """
SELECT DISTINCT ?pha ?p_name ?c ?c_name
WHERE {
   # bind something
   ?pha wdt:P97 wd:Q37110 .
   ?pha wdt:P40 ?c .
   # get the label
   ?pha sc:name ?p_name.
   ?c sc:name ?c_name.
}
Order BY ?pha ?p_name
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('pha', 'http://www.wikidata.org/entity/Q106771'), ('p_name', 'Hor-Aha'), ('c', 'http://www.wikidata.org/entity/Q152375'), ('c_name', 'Djer')]
[('pha', 'http://www.wikidata.org/entity/Q111079'), ('p_name', 'Anedjib'), ('c', 'http://www.wikidata.org/entity/Q151805'), ('c_name', 'Semerkhet')]
[('pha', 'http://www.wikidata.org/entity/Q125102'), ('p_name', 'Necho II'), ('c', 'http://www.wikidata.org/entity/Q373521'), ('c_name', 'Psamtik II')]
[('pha', 'http://www.wikidata.org/entity/Q129165'), ('p_name', 'Xerxes I'), ('c', 'http://www.wikidata.org/entity/Q15804594'), ('c_name', 'Darius')]
[('pha', 'http://www.wikidata.org/entity/Q129165'), ('p_name', 'Xerxes I'), ('c', 'http://www.wikidata.org/entity/Q189689'), ('c_name', 'Artaxerxes I of Persia')]
[('pha', 'http://www.wikidata.org/entity/Q129165'), ('p_name', 'Xerxes I'), ('c', 'http://www.wikidata.org/entity/Q2917844'), ('c_name', 'Rodoguna')]
[('pha', 'http://www.wikidata.org/entity/Q129165'), ('p_name', 'Xerxes I'), ('c', 'htt

In [13]:
# Check query
queryString = """
SELECT DISTINCT ?c ?c_name
WHERE {
   # bind something
   wd:Q199861 wdt:P40 ?c .
   # get the label
   ?c sc:name ?c_name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('c', 'http://www.wikidata.org/entity/Q200986'), ('c_name', 'Menkaure')]
[('c', 'http://www.wikidata.org/entity/Q456700'), ('c_name', 'Khamerernebty II')]
[('c', 'http://www.wikidata.org/entity/Q1250015'), ('c_name', 'Duaenre')]
[('c', 'http://www.wikidata.org/entity/Q6095558'), ('c_name', 'Iunre')]
[('c', 'http://www.wikidata.org/entity/Q769482'), ('c_name', 'Nebemakhet')]
[('c', 'http://www.wikidata.org/entity/Q84959355'), ('c_name', 'Shepsetkau')]
[('c', 'http://www.wikidata.org/entity/Q364562'), ('c_name', 'Iunmin I')]
[('c', 'http://www.wikidata.org/entity/Q204235'), ('c_name', 'Nikaure')]
[('c', 'http://www.wikidata.org/entity/Q3577093'), ('c_name', 'Ankhmare')]
9


In [14]:
# "Old Kingdom of Egypt" detais
queryString = """
SELECT DISTINCT ?p ?name ?o ?o_name
WHERE {
   # bind something
   wd:Q177819 ?p ?o .
   # get the label
   ?p sc:name ?name.
   ?o sc:name ?o_name.
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P156'), ('name', 'followed by'), ('o', 'http://www.wikidata.org/entity/Q232211'), ('o_name', 'First Intermediate Period of Egypt')]
[('p', 'http://www.wikidata.org/prop/direct/P36'), ('name', 'capital'), ('o', 'http://www.wikidata.org/entity/Q5715'), ('o_name', 'Memphis')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('name', 'part of'), ('o', 'http://www.wikidata.org/entity/Q766876'), ('o_name', 'history of ancient Egypt')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q11514315'), ('o_name', 'historical period')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q3024240'), ('o_name', 'historical country')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country'), ('o', 'http://www.wikidata.org/entity/Q79'), ('o_name', 'Egypt')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('n

In [15]:
# "Human" detais
queryString = """
SELECT DISTINCT ?p ?name ?o ?o_name
WHERE {
   # bind something
   wd:Q5 ?p ?o .
   # get the label
   ?p sc:name ?name.
   ?o sc:name ?o_name.
   FILTER regex(?name, "birth", "i")
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [16]:
# "Historical period" detais
queryString = """
SELECT DISTINCT ?p ?name ?o ?o_name
WHERE {
   # bind something
   wd:Q11514315 ?p ?o .
   # get the label
   ?p sc:name ?name.
   ?o sc:name ?o_name.
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P460'), ('name', 'said to be the same as'), ('o', 'http://www.wikidata.org/entity/Q816829'), ('o_name', 'periodization')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P580'), ('o_name', 'start time')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P582'), ('o_name', 'end time')]
[('p', 'http://www.wikidata.org/prop/direct/P8952'), ('name', 'inappropriate property for this type'), ('o', 'http://www.wikidata.org/entity/P576'), ('o_name', 'dissolved, abolished or demolished date')]
[('p', 'http://www.wikidata.org/prop/direct/P8952'), ('name', 'inappropriate property for this type'), ('o', 'http://www.wikidata.org/entity/P571'), ('o_name', 'inception')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q6428674'

In [17]:
# "Human" detais
queryString = """
SELECT DISTINCT ?p ?name ?o ?o_name
WHERE {
   # bind something
   wd:Q5 ?p ?o .
   # get the label
   ?p sc:name ?name.
   ?o sc:name ?o_name.
   FILTER regex(?o_name, "birth", "i")
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P19'), ('o_name', 'place of birth')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P1477'), ('o_name', 'birth name')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P569'), ('o_name', 'date of birth')]
3


In [18]:
# Sneferu date of birth
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   # bind something
   wd:Q189371 wdt:P569 ?o .
   # get the label
   ?o sc:name ?name.
}
LIMIT 40
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [19]:
# "Human" detais
queryString = """
SELECT DISTINCT ?p ?name ?o ?o_name
WHERE {
   # bind something
   wd:Q5 ?p ?o .
   # get the label
   ?p sc:name ?name.
   ?o sc:name ?o_name.
   
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P1317'), ('o_name', 'floruit')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P1477'), ('o_name', 'birth name')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P1559'), ('o_name', 'name in native language')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P509'), ('o_name', 'cause of death')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P3373'), ('o_name', 'sibling')]
[('p', 'http://www.wikidata.org/prop/direct/P1552'), ('name', 'has quality'), ('o', 'http://www.wikidata.org/entity/Q785260'), ('o_name', 'phase of human life')]


In [20]:
# Sneferu date of birth
queryString = """
SELECT DISTINCT ?d
WHERE {
   # bind something
   wd:Q189371 wdt:P569 ?d .
   # get the label
}
LIMIT 40
"""

print("Results")
x=run_query(queryString)

Results
[('d', '-2700-01-01T00:00:00Z')]
1


In [21]:
# Pharaos with more than 8 child born in the required frame
queryString = """
SELECT DISTINCT ?pha ?p_name (count(?c) as ?c_num)
WHERE {
   # bind something
   ?pha wdt:P97 wd:Q37110 ;
        wdt:P40 ?c ;
        wdt:P569 ?d ;
   # get the label
       sc:name ?p_name.
    
    FILTER (?d >= xsd:date("-1500-01-01") && ?d < xsd:date("-999-01-01")) .
}
GROUP BY ?pha ?p_name
HAVING(count(?c)>8)
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('pha', 'http://www.wikidata.org/entity/Q1528'), ('p_name', 'Ramesses III'), ('c_num', '10')]
[('pha', 'http://www.wikidata.org/entity/Q1523'), ('p_name', 'Ramesses II'), ('c_num', '18')]
[('pha', 'http://www.wikidata.org/entity/Q42606'), ('p_name', 'Amenhotep III'), ('c_num', '10')]
[('pha', 'http://www.wikidata.org/entity/Q81794'), ('p_name', 'Akhenaten'), ('c_num', '9')]
4


Final query for this task

In [22]:
# write your final query

queryString = """
SELECT DISTINCT ?pha ?p_name (count(?c) as ?c_num)
WHERE {
   # bind something
   ?pha wdt:P97 wd:Q37110 ;
        wdt:P40 ?c ;
        wdt:P569 ?d ;
   # get the label
       sc:name ?p_name.
    
    FILTER (?d >= xsd:date("-1500-01-01") && ?d < xsd:date("-999-01-01")) .
}
GROUP BY ?pha ?p_name
HAVING(count(?c)>8)
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('pha', 'http://www.wikidata.org/entity/Q1528'), ('p_name', 'Ramesses III'), ('c_num', '10')]
[('pha', 'http://www.wikidata.org/entity/Q1523'), ('p_name', 'Ramesses II'), ('c_num', '18')]
[('pha', 'http://www.wikidata.org/entity/Q42606'), ('p_name', 'Amenhotep III'), ('c_num', '10')]
[('pha', 'http://www.wikidata.org/entity/Q81794'), ('p_name', 'Akhenaten'), ('c_num', '9')]
4


## Task 3
Find all the ancient civilizations. Return the IRI and name of the civilization and also their duration (in number of years) if it is available.

In [23]:
# "Ancient Egypt" details.

queryString = """
SELECT DISTINCT ?p ?p_name ?o ?o_name
WHERE {
   # bind something
   wd:Q11768 ?p ?o . 
   # get the label
   ?p sc:name ?p_name.
   ?o sc:name ?o_name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('p_name', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q28171280'), ('o_name', 'ancient civilization')]
[('p', 'http://www.wikidata.org/prop/direct/P2184'), ('p_name', 'history of topic'), ('o', 'http://www.wikidata.org/entity/Q766876'), ('o_name', 'history of ancient Egypt')]
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('p_name', 'follows'), ('o', 'http://www.wikidata.org/entity/Q714601'), ('o_name', 'Predynastic Period of Egypt')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('p_name', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q1138524'), ('o_name', 'Paulys Realenzyklopädie der klassischen Altertumswissenschaft')]
[('p', 'http://www.wikidata.org/prop/direct/P30'), ('p_name', 'continent'), ('o', 'http://www.wikidata.org/entity/Q15'), ('o_name', 'Africa')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('p_name', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q1

In [24]:
# "ancient civilization" details.

queryString = """
SELECT DISTINCT ?p ?p_name ?o ?o_name
WHERE {
   # bind something
   wd:Q28171280 ?p ?o . 
   # get the label
   ?p sc:name ?p_name.
   ?o sc:name ?o_name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('p_name', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q8432'), ('o_name', 'civilization')]
1


In [25]:
# "civilization" details.

queryString = """
SELECT DISTINCT ?p ?p_name ?o ?o_name
WHERE {
   # bind somethingc
   wd:Q8432 ?p ?o . 
   # get the label
   ?p sc:name ?p_name.
   ?o sc:name ?o_name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('p_name', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q8425'), ('o_name', 'society')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('p_name', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q2041543'), ('o_name', 'Ottův slovník naučný')]
[('p', 'http://www.wikidata.org/prop/direct/P5008'), ('p_name', 'on focus list of Wikimedia project'), ('o', 'http://www.wikidata.org/entity/Q5460604'), ('o_name', 'Wikipedia:Vital articles')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('p_name', 'different from'), ('o', 'http://www.wikidata.org/entity/Q1868663'), ('o_name', 'Civilization')]
[('p', 'http://www.wikidata.org/prop/direct/P1269'), ('p_name', 'facet of'), ('o', 'http://www.wikidata.org/entity/Q1156970'), ('o_name', 'humanity')]
[('p', 'http://www.wikidata.org/prop/direct/P461'), ('p_name', 'opposite of'), ('o', 'http://www.wikidata.org/entity/Q83781188'), ('o_name', 'primitive culture')]
[

In [26]:
# "society" details.

queryString = """
SELECT DISTINCT ?p ?p_name ?o ?o_name
WHERE {
   # bind something
   wd:Q8425 ?p ?o . 
   # get the label
   ?p sc:name ?p_name.
   ?o sc:name ?o_name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1552'), ('p_name', 'has quality'), ('o', 'http://www.wikidata.org/entity/Q211606'), ('o_name', 'social structure')]
[('p', 'http://www.wikidata.org/prop/direct/P1552'), ('p_name', 'has quality'), ('o', 'http://www.wikidata.org/entity/Q3307505'), ('o_name', 'sociality')]
[('p', 'http://www.wikidata.org/prop/direct/P1552'), ('p_name', 'has quality'), ('o', 'http://www.wikidata.org/entity/Q345367'), ('o_name', 'social')]
[('p', 'http://www.wikidata.org/prop/direct/P1552'), ('p_name', 'has quality'), ('o', 'http://www.wikidata.org/entity/Q1510761'), ('o_name', 'social change')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('p_name', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q151885'), ('o_name', 'concept')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('p_name', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q19190511'), ('o_name', 'New Encyclopedic Dictionary')]
[('p', 'http://www.wikidata.org/pr

In [40]:
# ancient civilizations.

queryString = """
SELECT DISTINCT ?s ?name
WHERE {
   # bind something
   ?s wdt:P31 wd:Q28171280 . 
   # get the label
   ?s sc:name ?name.
}
LIMIT 25
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q35355'), ('name', 'Sumer')]
[('s', 'http://www.wikidata.org/entity/Q25558247'), ('name', 'Chinese civilization')]
[('s', 'http://www.wikidata.org/entity/Q38060'), ('name', 'Gaul')]
[('s', 'http://www.wikidata.org/entity/Q134178'), ('name', 'Minoan civilization')]
[('s', 'http://www.wikidata.org/entity/Q173082'), ('name', 'Dacia')]
[('s', 'http://www.wikidata.org/entity/Q2429397'), ('name', 'Ancient Carthage')]
[('s', 'http://www.wikidata.org/entity/Q3327860'), ('name', 'Celtic kingdom of Ireland')]
[('s', 'http://www.wikidata.org/entity/Q193748'), ('name', 'history of ancient Israel and Judah')]
[('s', 'http://www.wikidata.org/entity/Q11768'), ('name', 'Ancient Egypt')]
[('s', 'http://www.wikidata.org/entity/Q11772'), ('name', 'Ancient Greece')]
[('s', 'http://www.wikidata.org/entity/Q41741'), ('name', 'Thrace')]
[('s', 'http://www.wikidata.org/entity/Q128904'), ('name', 'Elam')]
[('s', 'http://www.wikidata.org/entity/Q185068'), ('name', 

In [29]:
# "Ancient Greece" details.

queryString = """
SELECT DISTINCT ?p ?p_name ?o ?o_name
WHERE {
   # bind something
   wd:Q11772 ?p ?o . 
   # get the label
   ?p sc:name ?p_name.
   ?o sc:name ?o_name.
}
LIMIT 40
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P8744'), ('p_name', 'economy of topic'), ('o', 'http://www.wikidata.org/entity/Q2736823'), ('o_name', 'economy of ancient Greece')]
[('p', 'http://www.wikidata.org/prop/direct/P2579'), ('p_name', 'studied by'), ('o', 'http://www.wikidata.org/entity/Q841090'), ('o_name', 'classics')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('p_name', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q28171280'), ('o_name', 'ancient civilization')]
[('p', 'http://www.wikidata.org/prop/direct/P2184'), ('p_name', 'history of topic'), ('o', 'http://www.wikidata.org/entity/Q7798'), ('o_name', 'history of ancient Greece')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('p_name', "topic's main category"), ('o', 'http://www.wikidata.org/entity/Q7215882'), ('o_name', 'Category:Ancient Greece')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('p_name', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q11042'), ('o_name', 'culture')]

In [36]:
# ancient civilizations duration?

queryString = """
SELECT DISTINCT ?p ?name ?s ?o
WHERE {
   # bind something
   ?s wdt:P31 wd:Q28171280 ;
       ?p ?o .
   # get the label
   ?p sc:name ?name.
   FILTER regex(?name, "duration", "i")
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [37]:
# ancient civilizations duration?

queryString = """
SELECT DISTINCT ?p ?name ?s ?o
WHERE {
   # bind something
   ?s wdt:P31 wd:Q28171280 ;
       ?p ?o .
   # get the label
   ?p sc:name ?name.
   FILTER regex(?name, "beginning", "i")
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [38]:
# ancient civilizations duration?

queryString = """
SELECT DISTINCT ?p ?name ?s ?o
WHERE {
   # bind something
   ?s wdt:P31 wd:Q28171280 ;
       ?p ?o .
   # get the label
   ?p sc:name ?name.
   FILTER regex(?name, "end", "i")
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P582'), ('name', 'end time'), ('s', 'http://www.wikidata.org/entity/Q134178'), ('o', '-1099-01-01T00:00:00Z')]
[('p', 'http://www.wikidata.org/prop/direct/P582'), ('name', 'end time'), ('s', 'http://www.wikidata.org/entity/Q3327860'), ('o', '1542-01-01T00:00:00Z')]
[('p', 'http://www.wikidata.org/prop/direct/P582'), ('name', 'end time'), ('s', 'http://www.wikidata.org/entity/Q11768'), ('o', '0395-01-01T00:00:00Z')]
[('p', 'http://www.wikidata.org/prop/direct/P582'), ('name', 'end time'), ('s', 'http://www.wikidata.org/entity/Q11772'), ('o', '0600-01-01T00:00:00Z')]
[('p', 'http://www.wikidata.org/prop/direct/P582'), ('name', 'end time'), ('s', 'http://www.wikidata.org/entity/Q42534'), ('o', '-1799-01-01T00:00:00Z')]
[('p', 'http://www.wikidata.org/prop/direct/P582'), ('name', 'end time'), ('s', 'http://www.wikidata.org/entity/Q1551054'), ('o', '-585-01-01T00:00:00Z')]
6


In [39]:
# ancient civilizations duration?

queryString = """
SELECT DISTINCT ?p ?name ?s ?s_name ?o
WHERE {
   # bind something
   ?s wdt:P31 wd:Q28171280 ;
       ?p ?o .
   # get the label
   ?p sc:name ?name.
   ?s sc:name ?s_name.
   FILTER regex(?name, "start", "i")
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P580'), ('name', 'start time'), ('s', 'http://www.wikidata.org/entity/Q134178'), ('s_name', 'Minoan civilization'), ('o', '-3299-01-01T00:00:00Z')]
[('p', 'http://www.wikidata.org/prop/direct/P580'), ('name', 'start time'), ('s', 'http://www.wikidata.org/entity/Q134178'), ('s_name', 'Minoan civilization'), ('o', '-2000-01-01T00:00:00Z')]
[('p', 'http://www.wikidata.org/prop/direct/P580'), ('name', 'start time'), ('s', 'http://www.wikidata.org/entity/Q11768'), ('s_name', 'Ancient Egypt'), ('o', '-4000-01-01T00:00:00Z')]
[('p', 'http://www.wikidata.org/prop/direct/P580'), ('name', 'start time'), ('s', 'http://www.wikidata.org/entity/Q11772'), ('s_name', 'Ancient Greece'), ('o', '-1200-01-01T00:00:00Z')]
[('p', 'http://www.wikidata.org/prop/direct/P580'), ('name', 'start time'), ('s', 'http://www.wikidata.org/entity/Q42534'), ('s_name', 'Indus Valley Civilization'), ('o', '-2999-01-01T00:00:00Z')]
[('p', 'http://www.wikidata.org/prop/dir

In [41]:
# ancient civilizations start time and end time

queryString = """
SELECT DISTINCT ?s ?s_name ?st ?et
WHERE {
   # bind something
   ?s wdt:P31 wd:Q28171280 .
   OPTIONAL {
       ?s wdt:P580 ?st
   } .
   OPTIONAL {
       ?s wdt:P582 ?et
   } .
   # get the label
   ?s sc:name ?s_name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q134178'), ('s_name', 'Minoan civilization'), ('st', '-3299-01-01T00:00:00Z'), ('et', '-1099-01-01T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q134178'), ('s_name', 'Minoan civilization'), ('st', '-2000-01-01T00:00:00Z'), ('et', '-1099-01-01T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q11768'), ('s_name', 'Ancient Egypt'), ('st', '-4000-01-01T00:00:00Z'), ('et', '0395-01-01T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q11772'), ('s_name', 'Ancient Greece'), ('st', '-1200-01-01T00:00:00Z'), ('et', '0600-01-01T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q42534'), ('s_name', 'Indus Valley Civilization'), ('st', '-2999-01-01T00:00:00Z'), ('et', '-1799-01-01T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q1551054'), ('s_name', 'Israelite period'), ('st', '-1199-01-01T00:00:00Z'), ('et', '-585-01-01T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q3327860'), ('s_name', 'Celtic kingdom of Ireland'), ('et', 

In [48]:
# ancient civilizations duration

queryString = """
SELECT DISTINCT ?s ?s_name ?t
WHERE {
   # bind something
   ?s wdt:P31 wd:Q28171280 .
   OPTIONAL {
       ?s wdt:P580 ?st
   } .
   OPTIONAL {
       ?s wdt:P582 ?et
   } .
   BIND( if(BOUND(?st) && BOUND(?et),(year(?et) - year(?st)),'') as ?t ) .
   # get the label
   ?s sc:name ?s_name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q134178'), ('s_name', 'Minoan civilization'), ('t', '2200')]
[('s', 'http://www.wikidata.org/entity/Q134178'), ('s_name', 'Minoan civilization'), ('t', '901')]
[('s', 'http://www.wikidata.org/entity/Q11768'), ('s_name', 'Ancient Egypt'), ('t', '4395')]
[('s', 'http://www.wikidata.org/entity/Q11772'), ('s_name', 'Ancient Greece'), ('t', '1800')]
[('s', 'http://www.wikidata.org/entity/Q42534'), ('s_name', 'Indus Valley Civilization'), ('t', '1200')]
[('s', 'http://www.wikidata.org/entity/Q1551054'), ('s_name', 'Israelite period'), ('t', '614')]
[('s', 'http://www.wikidata.org/entity/Q3327860'), ('s_name', 'Celtic kingdom of Ireland'), ('t', '')]
[('s', 'http://www.wikidata.org/entity/Q35355'), ('s_name', 'Sumer'), ('t', '')]
[('s', 'http://www.wikidata.org/entity/Q25558247'), ('s_name', 'Chinese civilization'), ('t', '')]
[('s', 'http://www.wikidata.org/entity/Q38060'), ('s_name', 'Gaul'), ('t', '')]
[('s', 'http://www.wikidata.org/entity/Q1

Final query for this task

In [49]:
# write your final query
# NOTE: The data seems incomplete and there are duplicates
queryString = """
SELECT DISTINCT ?s ?s_name ?t
WHERE {
   # bind something
   ?s wdt:P31 wd:Q28171280 .
   OPTIONAL {
       ?s wdt:P580 ?st
   } .
   OPTIONAL {
       ?s wdt:P582 ?et
   } .
   BIND( if(BOUND(?st) && BOUND(?et),(year(?et) - year(?st)),'') as ?t ) .
   # get the label
   ?s sc:name ?s_name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q134178'), ('s_name', 'Minoan civilization'), ('t', '2200')]
[('s', 'http://www.wikidata.org/entity/Q134178'), ('s_name', 'Minoan civilization'), ('t', '901')]
[('s', 'http://www.wikidata.org/entity/Q11768'), ('s_name', 'Ancient Egypt'), ('t', '4395')]
[('s', 'http://www.wikidata.org/entity/Q11772'), ('s_name', 'Ancient Greece'), ('t', '1800')]
[('s', 'http://www.wikidata.org/entity/Q42534'), ('s_name', 'Indus Valley Civilization'), ('t', '1200')]
[('s', 'http://www.wikidata.org/entity/Q1551054'), ('s_name', 'Israelite period'), ('t', '614')]
[('s', 'http://www.wikidata.org/entity/Q3327860'), ('s_name', 'Celtic kingdom of Ireland'), ('t', '')]
[('s', 'http://www.wikidata.org/entity/Q35355'), ('s_name', 'Sumer'), ('t', '')]
[('s', 'http://www.wikidata.org/entity/Q25558247'), ('s_name', 'Chinese civilization'), ('t', '')]
[('s', 'http://www.wikidata.org/entity/Q38060'), ('s_name', 'Gaul'), ('t', '')]
[('s', 'http://www.wikidata.org/entity/Q1

## Task 4
Identify the BGP for archeologist.

In [51]:
# archeologist profession

queryString = """
SELECT DISTINCT ?o ?o_name
WHERE {
   # bind something
   ?s wdt:P106 ?o . 
   # get the label
   ?o sc:name ?o_name.
   FILTER regex(?o_name, "archeo", "i")
}
LIMIT 40
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q83414831'), ('o_name', 'bioarcheologist')]
[('o', 'http://www.wikidata.org/entity/Q62061006'), ('o_name', 'maritime archeologist')]
[('o', 'http://www.wikidata.org/entity/Q97011145'), ('o_name', 'archeological scientist')]
3


In [52]:

queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q13444093 wdt:P31 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q1456099'), ('name', 'step pyramid')]
1


In [53]:

queryString = """
SELECT DISTINCT ?obj ?name ?p ?p_name
WHERE {
   # bind something
   wd:Q1456099 ?p ?obj .
   # get the label
   ?p sc:name ?p_name.
   ?obj sc:name ?name.
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q4363262'), ('name', 'smooth-sided pyramid'), ('p', 'http://www.wikidata.org/prop/direct/P461'), ('p_name', 'opposite of')]
[('obj', 'http://www.wikidata.org/entity/Q12516'), ('name', 'pyramid'), ('p', 'http://www.wikidata.org/prop/direct/P279'), ('p_name', 'subclass of')]
2


In [54]:
# pyramid details
queryString = """
SELECT DISTINCT ?obj ?name ?p ?p_name
WHERE {
   # bind something
   wd:Q12516 ?p ?obj .
   # get the label
   ?p sc:name ?p_name.
   ?obj sc:name ?name.
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q106727050'), ('name', 'Metropolitan Museum of Art Tagging Vocabulary'), ('p', 'http://www.wikidata.org/prop/direct/P1343'), ('p_name', 'described by source')]
[('obj', 'http://www.wikidata.org/entity/Q3181656'), ('name', 'The Nuttall Encyclopædia'), ('p', 'http://www.wikidata.org/prop/direct/P1343'), ('p_name', 'described by source')]
[('obj', 'http://www.wikidata.org/entity/Q39297'), ('name', 'volume'), ('p', 'http://www.wikidata.org/prop/direct/P1552'), ('p_name', 'has quality')]
[('obj', 'http://www.wikidata.org/entity/Q5460604'), ('name', 'Wikipedia:Vital articles'), ('p', 'http://www.wikidata.org/prop/direct/P5008'), ('p_name', 'on focus list of Wikimedia project')]
[('obj', 'http://www.wikidata.org/entity/Q867541'), ('name', 'Encyclopædia Britannica 11th edition'), ('p', 'http://www.wikidata.org/prop/direct/P1343'), ('p_name', 'described by source')]
[('obj', 'http://www.wikidata.org/entity/Q41176'), ('name', 'building'), ('p', 'h

In [55]:
# 'archeological scientist' details
queryString = """
SELECT DISTINCT ?obj ?name ?p ?p_name
WHERE {
   # bind something
   wd:Q97011145 ?p ?obj .
   # get the label
   ?p sc:name ?p_name.
   ?obj sc:name ?name.
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q15319501'), ('name', 'social scientist'), ('p', 'http://www.wikidata.org/prop/direct/P279'), ('p_name', 'subclass of')]
[('obj', 'http://www.wikidata.org/entity/Q19829999'), ('name', 'cultural studies scholar'), ('p', 'http://www.wikidata.org/prop/direct/P279'), ('p_name', 'subclass of')]
[('obj', 'http://www.wikidata.org/entity/Q12737077'), ('name', 'occupation'), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('p_name', 'instance of')]
[('obj', 'http://www.wikidata.org/entity/Q28640'), ('name', 'profession'), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('p_name', 'instance of')]
[('obj', 'http://www.wikidata.org/entity/Q637284'), ('name', 'archaeological science'), ('p', 'http://www.wikidata.org/prop/direct/P425'), ('p_name', 'field of this occupation')]
5


In [56]:
# 'archeological science' details
queryString = """
SELECT DISTINCT ?obj ?name ?p ?p_name
WHERE {
   # bind something
   wd:Q637284 ?p ?obj .
   # get the label
   ?p sc:name ?p_name.
   ?obj sc:name ?name.
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q4785472'), ('name', 'archaeological sub-discipline'), ('p', 'http://www.wikidata.org/prop/direct/P279'), ('p_name', 'subclass of')]
[('obj', 'http://www.wikidata.org/entity/Q15957611'), ('name', 'Category:Archaeological science'), ('p', 'http://www.wikidata.org/prop/direct/P910'), ('p_name', "topic's main category")]
[('obj', 'http://www.wikidata.org/entity/Q97011145'), ('name', 'archeological scientist'), ('p', 'http://www.wikidata.org/prop/direct/P3095'), ('p_name', 'practiced by')]
3


In [57]:
# 'archaeological sub-discipline' details
queryString = """
SELECT DISTINCT ?obj ?name ?p ?p_name
WHERE {
   # bind something
   wd:Q4785472 ?p ?obj .
   # get the label
   ?p sc:name ?p_name.
   ?obj sc:name ?name.
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q11862829'), ('name', 'academic discipline'), ('p', 'http://www.wikidata.org/prop/direct/P279'), ('p_name', 'subclass of')]
[('obj', 'http://www.wikidata.org/entity/Q5962346'), ('name', 'classification system'), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('p_name', 'instance of')]
[('obj', 'http://www.wikidata.org/entity/Q23498'), ('name', 'archaeology'), ('p', 'http://www.wikidata.org/prop/direct/P361'), ('p_name', 'part of')]
[('obj', 'http://www.wikidata.org/entity/Q24017414'), ('name', 'second-order class'), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('p_name', 'instance of')]
[('obj', 'http://www.wikidata.org/entity/Q6436272'), ('name', 'Category:Archaeological sub-disciplines'), ('p', 'http://www.wikidata.org/prop/direct/P910'), ('p_name', "topic's main category")]
[('obj', 'http://www.wikidata.org/entity/Q106225749'), ('name', 'archaeological sub-disciplines'), ('p', 'http://www.wikidata.org/prop/direct/P2354'), ('p_

In [58]:
# 'archaeology' details
queryString = """
SELECT DISTINCT ?obj ?name ?p ?p_name
WHERE {
   # bind something
   wd:Q23498 ?p ?obj .
   # get the label
   ?p sc:name ?p_name.
   ?obj sc:name ?name.
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q1066186'), ('name', 'study of history'), ('p', 'http://www.wikidata.org/prop/direct/P279'), ('p_name', 'subclass of')]
[('obj', 'http://www.wikidata.org/entity/Q11862829'), ('name', 'academic discipline'), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('p_name', 'instance of')]
[('obj', 'http://www.wikidata.org/entity/Q19180675'), ('name', 'Small Brockhaus and Efron Encyclopedic Dictionary'), ('p', 'http://www.wikidata.org/prop/direct/P1343'), ('p_name', 'described by source')]
[('obj', 'http://www.wikidata.org/entity/Q19211082'), ('name', 'Orthodox Theological Encyclopedia'), ('p', 'http://www.wikidata.org/prop/direct/P1343'), ('p_name', 'described by source')]
[('obj', 'http://www.wikidata.org/entity/Q2657718'), ('name', 'Armenian Soviet Encyclopedia'), ('p', 'http://www.wikidata.org/prop/direct/P1343'), ('p_name', 'described by source')]
[('obj', 'http://www.wikidata.org/entity/Q3181656'), ('name', 'The Nuttall Encyclopædia'), ('

Final query for this task

In [59]:
# write your final query

queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q23498 wdt:P3095 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q3621491'), ('name', 'archaeologist')]
1


## Task 5
For each ancient civilization find the number of archeologists who worked on it (return also the civilizations with no archeologists) (the result set must be triples of civilization IRI, label and #archeologist).

In [3]:
# Things related to archeologist

queryString = """
SELECT DISTINCT ?s ?name ?p ?p_name
WHERE {
   # bind something
   ?s ?p wd:Q3621491 .
   # get the label
   ?s sc:name ?name.
   ?p sc:name ?p_name.
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q364239'), ('name', 'William Camden'), ('p', 'http://www.wikidata.org/prop/direct/P101'), ('p_name', 'field of work')]
[('s', 'http://www.wikidata.org/entity/Q9176125'), ('name', 'Bohdan Janusz'), ('p', 'http://www.wikidata.org/prop/direct/P101'), ('p_name', 'field of work')]
[('s', 'http://www.wikidata.org/entity/Q102139'), ('name', 'Margrethe II of Denmark'), ('p', 'http://www.wikidata.org/prop/direct/P101'), ('p_name', 'field of work')]
[('s', 'http://www.wikidata.org/entity/Q20609875'), ('name', 'Numan Negmatov'), ('p', 'http://www.wikidata.org/prop/direct/P101'), ('p_name', 'field of work')]
[('s', 'http://www.wikidata.org/entity/Q1677608'), ('name', 'Jacob Pinkerfield'), ('p', 'http://www.wikidata.org/prop/direct/P101'), ('p_name', 'field of work')]
[('s', 'http://www.wikidata.org/entity/Q1821977'), ('name', 'Lev Perovski'), ('p', 'http://www.wikidata.org/prop/direct/P101'), ('p_name', 'field of work')]
[('s', 'http://www.wikidata.or

In [4]:
# Predicates related to archeologist

queryString = """
SELECT DISTINCT ?p ?p_name
WHERE {
   # bind something
   ?s ?p wd:Q3621491 .
   # get the label
   ?s sc:name ?name.
   ?p sc:name ?p_name.
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P101'), ('p_name', 'field of work')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('p_name', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('p_name', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P301'), ('p_name', "category's main topic")]
[('p', 'http://www.wikidata.org/prop/direct/P3095'), ('p_name', 'practiced by')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('p_name', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P360'), ('p_name', 'is a list of')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('p_name', 'part of')]
[('p', 'http://www.wikidata.org/prop/direct/P39'), ('p_name', 'position held')]
[('p', 'http://www.wikidata.org/prop/direct/P3989'), ('p_name', 'members have occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P971'), ('p_name', 'category combines topics')]
[('p', 'http://www.wikidata.org/prop/direct/P2925'), ('p_name', 'domain of saint or dei

In [5]:
# Subjects related to is a list of archeologist

queryString = """
SELECT DISTINCT ?s ?name
WHERE {
   # bind something
   ?s wdt:P360 wd:Q3621491 .
   # get the label
   ?s sc:name ?name.
   ?p sc:name ?p_name.
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q1492089'), ('name', 'list of archaeologists')]
1


In [6]:
# Things related to list of archeologist

queryString = """
SELECT DISTINCT ?s ?name ?p ?p_name
WHERE {
   # bind something
   ?s ?p wd:Q1492089 .
   # get the label
   ?s sc:name ?name.
   ?p sc:name ?p_name.
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q3621491'), ('name', 'archaeologist'), ('p', 'http://www.wikidata.org/prop/direct/P2354'), ('p_name', 'has list')]
1


In [3]:
# field of work of an archeologist

queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   # bind something
   wd:Q3839667 wdt:P101 ?o .
   # get the label
   ?o sc:name ?name.
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [5]:
# field of work of some archeologist WRONG

queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   # bind something
   ?a wdt:P106 wd:Q3621491 .
   wd:Q3839667 wdt:P101 ?o .
   # get the label
   ?o sc:name ?name.
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [6]:
# some archeologist

queryString = """
SELECT DISTINCT ?a ?name
WHERE {
   # bind something
   ?a wdt:P106 wd:Q3621491 .
   # get the label
   ?a sc:name ?name.
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('a', 'http://www.wikidata.org/entity/Q8463726'), ('name', 'Chengzuo Shang')]
[('a', 'http://www.wikidata.org/entity/Q13638340'), ('name', 'Dagmar Jelínková')]
[('a', 'http://www.wikidata.org/entity/Q15466790'), ('name', 'Josef Poppelreuter')]
[('a', 'http://www.wikidata.org/entity/Q15729291'), ('name', 'Marino Marini')]
[('a', 'http://www.wikidata.org/entity/Q15730811'), ('name', 'Hartmut Schäfer')]
[('a', 'http://www.wikidata.org/entity/Q15731701'), ('name', 'Pietro Tessieri')]
[('a', 'http://www.wikidata.org/entity/Q15733244'), ('name', 'Daring Do')]
[('a', 'http://www.wikidata.org/entity/Q3839667'), ('name', 'Luigi Gaetano Marini')]
[('a', 'http://www.wikidata.org/entity/Q104327'), ('name', 'Otto Rossbach')]
[('a', 'http://www.wikidata.org/entity/Q15906413'), ('name', 'Frank Henry Stubbings')]
[('a', 'http://www.wikidata.org/entity/Q5717733'), ('name', 'Henry Balfour')]
[('a', 'http://www.wikidata.org/entity/Q704501'), ('name', 'Sergey Uvarov')]
[('a', 'http://www.wikidata

In [7]:
# field of work of some archeologists

queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   # bind something
   ?a wdt:P106 wd:Q3621491 .
   ?a wdt:P101 ?o .
   # get the label
   ?o sc:name ?name.
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q1265179'), ('name', 'maritime archaeology')]
[('o', 'http://www.wikidata.org/entity/Q636481'), ('name', 'paleoethnobotany')]
[('o', 'http://www.wikidata.org/entity/Q213649'), ('name', 'Viking Age')]
[('o', 'http://www.wikidata.org/entity/Q219488'), ('name', 'primatology')]
[('o', 'http://www.wikidata.org/entity/Q2532901'), ('name', 'environmental archaeology')]
[('o', 'http://www.wikidata.org/entity/Q7573930'), ('name', 'megalithic art')]
[('o', 'http://www.wikidata.org/entity/Q1069'), ('name', 'geology')]
[('o', 'http://www.wikidata.org/entity/Q130436'), ('name', 'history of Japan')]
[('o', 'http://www.wikidata.org/entity/Q1569495'), ('name', 'lecturer')]
[('o', 'http://www.wikidata.org/entity/Q208217'), ('name', 'literary studies')]
[('o', 'http://www.wikidata.org/entity/Q37260'), ('name', 'writing')]
[('o', 'http://www.wikidata.org/entity/Q476294'), ('name', 'Oriental studies')]
[('o', 'http://www.wikidata.org/entity/Q4932206'), ('name

In [9]:
# field of work of some archeologists

queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   # bind something
   ?a wdt:P106 wd:Q3621491 .
   ?a wdt:P101 ?o .
   # get the label
   ?o sc:name ?name.
   FILTER regex(?name, "ancient", "i")
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q41493'), ('name', 'ancient history')]
[('o', 'http://www.wikidata.org/entity/Q7798'), ('name', 'history of ancient Greece')]
[('o', 'http://www.wikidata.org/entity/Q11772'), ('name', 'Ancient Greece')]
[('o', 'http://www.wikidata.org/entity/Q1747689'), ('name', 'Ancient Rome')]
[('o', 'http://www.wikidata.org/entity/Q269678'), ('name', 'ancient Near East')]
[('o', 'http://www.wikidata.org/entity/Q4246634'), ('name', 'Ancient Greek culture')]
[('o', 'http://www.wikidata.org/entity/Q2429397'), ('name', 'Ancient Carthage')]
[('o', 'http://www.wikidata.org/entity/Q337547'), ('name', 'ancient Roman religion')]
[('o', 'http://www.wikidata.org/entity/Q4752916'), ('name', 'Ancient Roman pottery')]
[('o', 'http://www.wikidata.org/entity/Q35966'), ('name', 'Ancient Celts')]
[('o', 'http://www.wikidata.org/entity/Q855270'), ('name', 'religion in ancient Greece')]
[('o', 'http://www.wikidata.org/entity/Q738680'), ('name', 'pottery of ancient Greece')

In [8]:
# archeologists working on Ancient Egypt

queryString = """
SELECT DISTINCT ?a ?name
WHERE {
   # bind something
   ?a wdt:P106 wd:Q3621491 .
   ?a wdt:P101 wd:Q11768 .
   # get the label
   ?a sc:name ?name.
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [10]:
# ancient civilizations and archeologists
queryString = """
SELECT DISTINCT ?ac ?name (count(distinct ?ar) as ?num_ar)
WHERE {
   # bind something
   ?ac wdt:P31 wd:Q28171280 . 
   ?ar wdt:P101 ?ac ;
       wdt:P106 wd:Q3621491 .
   # get the label
   ?ac sc:name ?name.
}
GROUP BY ?ac ?name
LIMIT 25
"""

print("Results")
x=run_query(queryString)

Results
[('ac', 'http://www.wikidata.org/entity/Q28567'), ('name', 'Maya civilization'), ('num_ar', '2')]
[('ac', 'http://www.wikidata.org/entity/Q41741'), ('name', 'Thrace'), ('num_ar', '1')]
[('ac', 'http://www.wikidata.org/entity/Q11767'), ('name', 'Mesopotamia'), ('num_ar', '1')]
[('ac', 'http://www.wikidata.org/entity/Q2429397'), ('name', 'Ancient Carthage'), ('num_ar', '7')]
[('ac', 'http://www.wikidata.org/entity/Q38060'), ('name', 'Gaul'), ('num_ar', '3')]
[('ac', 'http://www.wikidata.org/entity/Q193748'), ('name', 'history of ancient Israel and Judah'), ('num_ar', '1')]
[('ac', 'http://www.wikidata.org/entity/Q1551054'), ('name', 'Israelite period'), ('num_ar', '2')]
[('ac', 'http://www.wikidata.org/entity/Q11772'), ('name', 'Ancient Greece'), ('num_ar', '7')]
8


Final query for this task

In [11]:
# write your final query
# NOTE: A lot of data seems missing
queryString = """
SELECT DISTINCT ?ac ?name (count(distinct ?ar) as ?num_ar)
WHERE {
   # bind something
   ?ac wdt:P31 wd:Q28171280 . 
   optional {
       ?ar wdt:P101 ?ac ;
           wdt:P106 wd:Q3621491 .
   }
   # get the label
   ?ac sc:name ?name.
}
GROUP BY ?ac ?name
ORDER BY desc(?num_ar)
LIMIT 25
"""

print("Results")
x=run_query(queryString)

Results
[('ac', 'http://www.wikidata.org/entity/Q2429397'), ('name', 'Ancient Carthage'), ('num_ar', '7')]
[('ac', 'http://www.wikidata.org/entity/Q11772'), ('name', 'Ancient Greece'), ('num_ar', '7')]
[('ac', 'http://www.wikidata.org/entity/Q38060'), ('name', 'Gaul'), ('num_ar', '3')]
[('ac', 'http://www.wikidata.org/entity/Q28567'), ('name', 'Maya civilization'), ('num_ar', '2')]
[('ac', 'http://www.wikidata.org/entity/Q1551054'), ('name', 'Israelite period'), ('num_ar', '2')]
[('ac', 'http://www.wikidata.org/entity/Q41741'), ('name', 'Thrace'), ('num_ar', '1')]
[('ac', 'http://www.wikidata.org/entity/Q11767'), ('name', 'Mesopotamia'), ('num_ar', '1')]
[('ac', 'http://www.wikidata.org/entity/Q193748'), ('name', 'history of ancient Israel and Judah'), ('num_ar', '1')]
[('ac', 'http://www.wikidata.org/entity/Q128904'), ('name', 'Elam'), ('num_ar', '0')]
[('ac', 'http://www.wikidata.org/entity/Q32047'), ('name', 'Illyria'), ('num_ar', '0')]
[('ac', 'http://www.wikidata.org/entity/Q28405

## Task 6
Find the BGP for deity.

In [7]:
# 'Poseidon' details
queryString = """
SELECT DISTINCT ?obj ?name ?p ?p_name
WHERE {
   # bind something
   wd:Q41127 ?p ?obj .
   # get the label
   ?p sc:name ?p_name.
   ?obj sc:name ?name.
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q272006'), ('name', 'Antaeus'), ('p', 'http://www.wikidata.org/prop/direct/P40'), ('p_name', 'child')]
[('obj', 'http://www.wikidata.org/entity/Q1817912'), ('name', 'Hyrieus'), ('p', 'http://www.wikidata.org/prop/direct/P40'), ('p_name', 'child')]
[('obj', 'http://www.wikidata.org/entity/Q763268'), ('name', 'Alcyone'), ('p', 'http://www.wikidata.org/prop/direct/P451'), ('p_name', 'unmarried partner')]
[('obj', 'http://www.wikidata.org/entity/Q1469237'), ('name', 'Sciron'), ('p', 'http://www.wikidata.org/prop/direct/P40'), ('p_name', 'child')]
[('obj', 'http://www.wikidata.org/entity/Q192082'), ('name', 'Charybdis'), ('p', 'http://www.wikidata.org/prop/direct/P40'), ('p_name', 'child')]
[('obj', 'http://www.wikidata.org/entity/Q29654939'), ('name', 'Achaeus'), ('p', 'http://www.wikidata.org/prop/direct/P40'), ('p_name', 'child')]
[('obj', 'http://www.wikidata.org/entity/Q19800714'), ('name', 'Periboea'), ('p', 'http://www.wikidata.org/pro

In [8]:
# 'Poseidon' predicates
queryString = """
SELECT DISTINCT ?p ?p_name
WHERE {
   # bind something
   wd:Q41127 ?p ?obj .
   # get the label
   ?p sc:name ?p_name.
   ?obj sc:name ?name.
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P40'), ('p_name', 'child')]
[('p', 'http://www.wikidata.org/prop/direct/P451'), ('p_name', 'unmarried partner')]
[('p', 'http://www.wikidata.org/prop/direct/P22'), ('p_name', 'father')]
[('p', 'http://www.wikidata.org/prop/direct/P1441'), ('p_name', 'present in work')]
[('p', 'http://www.wikidata.org/prop/direct/P3373'), ('p_name', 'sibling')]
[('p', 'http://www.wikidata.org/prop/direct/P26'), ('p_name', 'spouse')]
[('p', 'http://www.wikidata.org/prop/direct/P2283'), ('p_name', 'uses')]
[('p', 'http://www.wikidata.org/prop/direct/P4185'), ('p_name', 'iconographic symbol')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('p_name', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P2925'), ('p_name', 'domain of saint or deity')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('p_name', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P460'), ('p_name', 'said to be the same as')]
[('p', 'http://www.

In [9]:
# 'Poseidon' is istance of
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q41127 wdt:P31 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q1916821'), ('name', 'water deity')]
[('obj', 'http://www.wikidata.org/entity/Q22989102'), ('name', 'Greek deity')]
2


In [10]:
# 'Greek deity' details
queryString = """
SELECT DISTINCT ?obj ?name ?p ?p_name
WHERE {
   # bind something
   wd:Q22989102 ?p ?obj .
   # get the label
   ?p sc:name ?p_name.
   ?obj sc:name ?name.
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q95074'), ('name', 'fictional character'), ('p', 'http://www.wikidata.org/prop/direct/P279'), ('p_name', 'subclass of')]
[('obj', 'http://www.wikidata.org/entity/Q11772'), ('name', 'Ancient Greece'), ('p', 'http://www.wikidata.org/prop/direct/P2596'), ('p_name', 'culture')]
[('obj', 'http://www.wikidata.org/entity/Q22988604'), ('name', 'mythological Greek character'), ('p', 'http://www.wikidata.org/prop/direct/P279'), ('p_name', 'subclass of')]
[('obj', 'http://www.wikidata.org/entity/Q178885'), ('name', 'deity'), ('p', 'http://www.wikidata.org/prop/direct/P279'), ('p_name', 'subclass of')]
[('obj', 'http://www.wikidata.org/entity/Q34726'), ('name', 'Greek mythology'), ('p', 'http://www.wikidata.org/prop/direct/P361'), ('p_name', 'part of')]
[('obj', 'http://www.wikidata.org/entity/Q855270'), ('name', 'religion in ancient Greece'), ('p', 'http://www.wikidata.org/prop/direct/P361'), ('p_name', 'part of')]
[('obj', 'http://www.wikidata.org

Final query for this task

In [11]:
# write your final query
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q22989102 wdt:P279 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q95074'), ('name', 'fictional character')]
[('obj', 'http://www.wikidata.org/entity/Q22988604'), ('name', 'mythological Greek character')]
[('obj', 'http://www.wikidata.org/entity/Q178885'), ('name', 'deity')]
3


## Task 7
List all the twelve olympian greek deities (the result set must a list of 12 elements of couples of deity IRI and label).

In [12]:
# 'Poseidon' is part of
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q41127 wdt:P361 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q101609'), ('name', 'Twelve Olympians')]
1


Final query for this task

In [13]:
# write your final query
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
   # bind something
   ?s wdt:P361 wd:Q101609 .
   # get the label
   ?s sc:name ?name.
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q37340'), ('name', 'Apollo')]
[('s', 'http://www.wikidata.org/entity/Q37122'), ('name', 'Athena')]
[('s', 'http://www.wikidata.org/entity/Q38012'), ('name', 'Hera')]
[('s', 'http://www.wikidata.org/entity/Q39503'), ('name', 'Artemis')]
[('s', 'http://www.wikidata.org/entity/Q44384'), ('name', 'Hephaestus')]
[('s', 'http://www.wikidata.org/entity/Q41484'), ('name', 'Hermes')]
[('s', 'http://www.wikidata.org/entity/Q41127'), ('name', 'Poseidon')]
[('s', 'http://www.wikidata.org/entity/Q41419'), ('name', 'Hestia')]
[('s', 'http://www.wikidata.org/entity/Q34201'), ('name', 'Zeus')]
[('s', 'http://www.wikidata.org/entity/Q35500'), ('name', 'Aphrodite')]
[('s', 'http://www.wikidata.org/entity/Q40730'), ('name', 'Demeter')]
[('s', 'http://www.wikidata.org/entity/Q40901'), ('name', 'Ares')]
12


## Task 8
Compare ancient Greece with ancient Egypt and find how many deities each one has.

In [2]:
# 'deity' details
queryString = """
SELECT DISTINCT ?obj ?name ?p ?p_name
WHERE {
   # bind something
   wd:Q178885 ?p ?obj .
   # get the label
   ?p sc:name ?p_name.
   ?obj sc:name ?name.
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q5648972'), ('name', 'Category:Deities'), ('p', 'http://www.wikidata.org/prop/direct/P910'), ('p_name', "topic's main category")]
[('obj', 'http://www.wikidata.org/entity/Q106727050'), ('name', 'Metropolitan Museum of Art Tagging Vocabulary'), ('p', 'http://www.wikidata.org/prop/direct/P1343'), ('p_name', 'described by source')]
[('obj', 'http://www.wikidata.org/entity/Q190'), ('name', 'God'), ('p', 'http://www.wikidata.org/prop/direct/P460'), ('p_name', 'said to be the same as')]
[('obj', 'http://www.wikidata.org/entity/Q105890973'), ('name', 'supernatural realm'), ('p', 'http://www.wikidata.org/prop/direct/P1269'), ('p_name', 'facet of')]
[('obj', 'http://www.wikidata.org/entity/Q17624054'), ('name', 'fictional deity'), ('p', 'http://www.wikidata.org/prop/direct/P1889'), ('p_name', 'different from')]
[('obj', 'http://www.wikidata.org/entity/Q4271324'), ('name', 'mythical character'), ('p', 'http://www.wikidata.org/prop/direct/P279'), (

In [3]:
# Subclasses of 'deity'
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
   # bind something
   ?s wdt:P279 wd:Q178885 .
   # get the label
   ?s sc:name ?name.
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q138840'), ('name', 'Common Germanic deities')]
[('s', 'http://www.wikidata.org/entity/Q642405'), ('name', 'dying-and-rising deity')]
[('s', 'http://www.wikidata.org/entity/Q1207451'), ('name', 'xian')]
[('s', 'http://www.wikidata.org/entity/Q3032311'), ('name', 'Native American deity')]
[('s', 'http://www.wikidata.org/entity/Q4089509'), ('name', 'Deva')]
[('s', 'http://www.wikidata.org/entity/Q94194127'), ('name', 'deity of mordvin pantheon')]
[('s', 'http://www.wikidata.org/entity/Q1470705'), ('name', 'fertility deity')]
[('s', 'http://www.wikidata.org/entity/Q1569856'), ('name', 'lunar deity')]
[('s', 'http://www.wikidata.org/entity/Q16513881'), ('name', 'Norse deity')]
[('s', 'http://www.wikidata.org/entity/Q41863069'), ('name', 'war deity')]
[('s', 'http://www.wikidata.org/entity/Q511056'), ('name', 'solar deity')]
[('s', 'http://www.wikidata.org/entity/Q6058157'), ('name', 'god')]
[('s', 'http://www.wikidata.org/entity/Q635155'), ('n

In [2]:
# Things related to 'Ancient Egyptian deity'
queryString = """
SELECT DISTINCT ?s ?name ?p ?p_name
WHERE {
   # bind something
   ?s ?p wd:Q146083 .
   # get the label
   ?s sc:name ?name.
   ?p sc:name ?p_name.
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q6573004'), ('name', 'Heliopolitans'), ('p', 'http://www.wikidata.org/prop/direct/P144'), ('p_name', 'based on')]
[('s', 'http://www.wikidata.org/entity/Q108092253'), ('name', 'Ancient Egyptian pantheon'), ('p', 'http://www.wikidata.org/prop/direct/P2670'), ('p_name', 'has parts of the class')]
[('s', 'http://www.wikidata.org/entity/Q48803719'), ('name', 'Cavern deities of the underworld'), ('p', 'http://www.wikidata.org/prop/direct/P279'), ('p_name', 'subclass of')]
[('s', 'http://www.wikidata.org/entity/Q48803736'), ('name', 'Gate deities of the underworld'), ('p', 'http://www.wikidata.org/prop/direct/P279'), ('p_name', 'subclass of')]
[('s', 'http://www.wikidata.org/entity/Q9777847'), ('name', 'Category:Egyptian deities'), ('p', 'http://www.wikidata.org/prop/direct/P301'), ('p_name', "category's main topic")]
[('s', 'http://www.wikidata.org/entity/Q7214264'), ('name', 'Category:Egyptian gods'), ('p', 'http://www.wikidata.org/prop/direct

In [12]:
# num of istances of 'Ancient Egyptian deity'
queryString = """
SELECT (count(DISTINCT ?s) as ?num)
WHERE {
   # bind something
   ?s wdt:P31 wd:Q146083 .
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('num', '281')]
1


In [13]:
# num of istances of 'Greek deity'
queryString = """
SELECT (count(DISTINCT ?s) as ?num)
WHERE {
   # bind something
   ?s wdt:P31 wd:Q22989102 .
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('num', '252')]
1


In [14]:
# check num of istances of 'Greek deity'
queryString = """
SELECT ?s ?name
WHERE {
   # bind something
   ?s wdt:P31 wd:Q22989102 .
   ?s sc:name ?name.
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q5081127'), ('name', 'Acmon')]
[('s', 'http://www.wikidata.org/entity/Q366533'), ('name', 'Adranus')]
[('s', 'http://www.wikidata.org/entity/Q3362052'), ('name', 'Pandora')]
[('s', 'http://www.wikidata.org/entity/Q6612'), ('name', 'Charon')]
[('s', 'http://www.wikidata.org/entity/Q2144233'), ('name', 'Pheme')]
[('s', 'http://www.wikidata.org/entity/Q1120723'), ('name', 'Pheme')]
[('s', 'http://www.wikidata.org/entity/Q1862189'), ('name', 'Paean')]
[('s', 'http://www.wikidata.org/entity/Q1775444'), ('name', 'Phosphorus')]
[('s', 'http://www.wikidata.org/entity/Q178417'), ('name', 'Eos')]
[('s', 'http://www.wikidata.org/entity/Q59697'), ('name', 'Hesperus')]
[('s', 'http://www.wikidata.org/entity/Q1084868'), ('name', 'Lyssa')]
[('s', 'http://www.wikidata.org/entity/Q1294659'), ('name', 'Telesphorus')]
[('s', 'http://www.wikidata.org/entity/Q18423351'), ('name', 'Basileia')]
[('s', 'http://www.wikidata.org/entity/Q185670'), ('name', 'Echidna'

Final query for this task

In [2]:
# write your final query
# NOTE: The number of Egyptian deities is clearly wrong
queryString = """
SELECT ?eg_num ?gr_num
WHERE {
   # bind something
   {
       SELECT (count(DISTINCT ?s) as ?eg_num)
        WHERE {
           # bind something
           ?s wdt:P31 wd:Q146083 .
        }
    } .
   {
       SELECT (count(DISTINCT ?s) as ?gr_num)
        WHERE {
           # bind something
           ?s wdt:P31 wd:Q22989102 .
        }
    }
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('eg_num', '281'), ('gr_num', '252')]
1
